# Build a variational autoencoder

### References

https://blog.keras.io/building-autoencoders-in-keras.html

https://towardsdatascience.com/understanding-variational-autoencoders-vaes-f70510919f73

In [1]:
import sys
import numpy as np
import keras
from keras import regularizers
from keras import layers
from keras import backend
import random

if '..' not in sys.path:
    sys.path.append('..')
from car_reg_generator.car_reg_generator.uk_reg import UkRegGenerator
from car_reg_generator.car_reg_generator.uk_reg import UkRegBowVectorizer

### Data generation

In [2]:
n_train = 10000
n_test = 1000

random.seed(0)
g = UkRegGenerator()
v = UkRegBowVectorizer()

train_strs = [g.get_reg()[2:] for _ in range(n_train)]
train_vecs = np.array([v.vectorize(x) for x in train_strs])
test_strs = [g.get_reg()[2:] for _ in range(n_test)]
test_vecs = np.array([v.vectorize(x) for x in test_strs])

vec_length = len(train_vecs[0])
print(vec_length)

180


### Variational autoencoder

In [3]:
latent_dim = 100
intermediate_dim = 300
original_dim = vec_length

### Define the encoder
inputs = keras.Input(shape=(original_dim,))
h = layers.Dense(intermediate_dim, activation='relu')(inputs)
h = layers.Dense(intermediate_dim, activation='relu')(h)
z_mean = layers.Dense(latent_dim)(h)
z_log_sigma = layers.Dense(latent_dim)(h)

def sampling(args):
    z_mean, z_log_sigma = args
    epsilon = backend.random_normal(shape=(backend.shape(z_mean)[0], latent_dim),
                                    mean=0., stddev=0.1)
    return z_mean + backend.exp(z_log_sigma) * epsilon

z = layers.Lambda(sampling)([z_mean, z_log_sigma])

encoder = keras.Model(inputs, [z_mean, z_log_sigma, z], name='encoder')

### Define the decoder
latent_inputs = keras.Input(shape=(latent_dim,), name='z_sampling')
x = layers.Dense(intermediate_dim, activation='relu')(latent_inputs)
# x = layers.Dense(intermediate_dim, activation='relu')(x)
outputs = layers.Dense(original_dim, activation='sigmoid')(x)
decoder = keras.Model(latent_inputs, outputs, name='decoder')

outputs = decoder(encoder(inputs)[2])
vae = keras.Model(inputs, outputs, name='vae_mlp')

### Define the loss function
reconstruction_loss = keras.losses.binary_crossentropy(inputs, outputs)
reconstruction_loss *= original_dim
kl_loss = 1 + z_log_sigma - backend.square(z_mean) - backend.exp(z_log_sigma)
kl_loss = backend.sum(kl_loss, axis=-1)
kl_loss *= -0.5
vae_loss = backend.mean(reconstruction_loss + kl_loss)
vae.add_loss(vae_loss)
vae.compile(optimizer='adam')

In [4]:
vae.fit(train_vecs, train_vecs,
        epochs=50,
        batch_size=256,
        shuffle=True,
        validation_data=(test_vecs, test_vecs))

Epoch 1/50
40/40 [==============================] - 2s 40ms/step - loss: 104.7476 - val_loss: 86.7640
Epoch 2/50
40/40 [==============================] - 1s 36ms/step - loss: 78.4095 - val_loss: 71.9424
Epoch 3/50
40/40 [==============================] - 1s 32ms/step - loss: 68.2727 - val_loss: 64.9099
Epoch 4/50
40/40 [==============================] - 1s 33ms/step - loss: 62.5828 - val_loss: 60.4019
Epoch 5/50
40/40 [==============================] - 1s 35ms/step - loss: 58.6618 - val_loss: 57.0134
Epoch 6/50
40/40 [==============================] - 1s 34ms/step - loss: 55.3630 - val_loss: 53.9075
Epoch 7/50
40/40 [==============================] - 1s 35ms/step - loss: 52.7265 - val_loss: 51.5895
Epoch 8/50
40/40 [==============================] - 1s 35ms/step - loss: 50.6263 - val_loss: 49.6916
Epoch 9/50
40/40 [==============================] - 2s 42ms/step - loss: 48.9272 - val_loss: 48.0632
Epoch 10/50
40/40 [==============================] - 2s 38ms/step - loss: 47.4134 - val_lo

In [5]:
decoded_regs = vae.predict(test_vecs)
print(test_strs[:10])
print([v.recover(x) for x in decoded_regs[:10]])
acc = np.sum([v.recover(x) == y for x, y in zip(decoded_regs, test_strs)]) / len(test_strs)
print('accuracy = ' + str(acc))

['45FVI', '23DCR', '99JWO', '58XKI', '49FNK', '61XIM', '62NON', '42JCE', '47KTZ', '70DFU']
['45FVI', '23DCR', '99JWO', '58QKI', '49FNK', '61XIM', '62NON', '42J3E', '47KTZ', '70DFU']
accuracy = 0.855
